In [4]:
import nest
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import time
import functions_nest as fn
nest.ResetKernel()
resolution = 0.1
nest.SetKernelStatus({
    "resolution": resolution,   
    "print_time": True
})
np.random.seed(42)


Jan 26 17:06:49 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.


# Input Architecture 

In [7]:
# ==================== Neuron Parameters =================
neuron_params = {
    "C_m": 200.0,        # pF (equivale a tau=20 ms con gL=10 nS)
    "g_L": 10.0,         # nS
    "E_L": -60.0,        # mV
    "V_th": -50.0,       # mV
    "V_reset": -60.0,    # mV
    "t_ref": 5.0,        # ms
    "E_ex": 0.0,         # mV
    "E_in": -80.0        # mV
}

# Synapse time constants
tau_exc = 5.0    # ms (τ_E = 5 ms)
tau_inh = 10.0   # ms (τ_I = 10 ms)

# Channel Parameters
n_groups = 8                    
N_exc_per_group = 100           
N_inh_per_group = 25  


# ==================== Create Signal Modulation ====================

T_sim = 20000.0  # ms
dt_signal = 1.0  # ms
tau_signal = 50.0  # ms
times = np.arange(dt_signal, T_sim + dt_signal, dt_signal)
alpha = dt_signal / tau_signal
t_now = nest.GetKernelStatus("biological_time")

print(t_now)
rate_times = t_now + 1e-6 + times
print(rate_times)

rate_mean = 10.0  # Hz
rate_std = 5.0    # Hz

signals = []
for g in range(n_groups):
    noise = np.random.randn(len(times))
    signal = np.zeros_like(noise)

    for i in range(1, len(signal)):
        signal[i] = signal[i-1] + alpha * (-signal[i-1] + noise[i])

    signal = (signal - np.mean(signal)) / np.std(signal)
    rates = rate_mean + rate_std * signal
    rates[rates < 0.1] = 0.1
    
    signals.append(rates)

delay = 1.5  # ms

0.0
[1.000001e+00 2.000001e+00 3.000001e+00 ... 1.999800e+04 1.999900e+04
 2.000000e+04]


In [10]:
# ==================== Create Post Neuron ====================

post = fn.post_neuron("aeif_cond_exp", tau_exc, tau_inh, n=1, neuron_params=neuron_params)


# ==================== Connection Parameters  ====================

# Parameters for inhibitory synapses (Vogels-Sprekeler)

inh_syn_params = {
    "synapse_model": "vogels_sprekeler_synapse",
    "weight": -0.1,      # Peso inicial DÉBIL (importante)
    "delay": delay,
    "eta": 0.0001,        # Tasa de aprendizaje
    "alpha": 0.2,      # Ratio
    "tau": 20.0,        # ms (constante de tiempo STDP)
    "Wmax": -100.0,      # Peso máximo
}

# Parameters for excitatory synapses (fixed weights)

preferred_group = 4  # Grupo 5 (0-based index 4)
w_exc_max = 3.0   # nS para grupo preferido
w_exc_min = 1.5     # nS para grupos no preferidos

w_exc_by_group = fn.exc_weights(n_groups, preferred_group)


# ==================== Create Connections ====================

for g in range(n_groups):

    # Excitatory connections

    parrots = nest.Create("parrot_neuron", N_exc_per_group)
    exc_gen = nest.Create("inhomogeneous_poisson_generator", params={
    "rate_times": rate_times.tolist(),
    "rate_values": signals[g].tolist()
     })
    

    nest.Connect(exc_gen, parrots, conn_spec="all_to_all")
    nest.Connect(parrots, post, conn_spec="all_to_all",
    syn_spec={
        "synapse_model": "static_synapse",
        "weight": w_exc_by_group[g],
        "delay": delay,
       # "receptor_type": 1 # Excitatory receptor
    })

   # Inhibitory connections

    parrots = nest.Create("parrot_neuron", N_inh_per_group)
    ihn_gen = nest.Create("inhomogeneous_poisson_generator", params={
    "rate_times": rate_times.tolist(),
    "rate_values": signals[g].tolist()
     })
    

    nest.Connect(ihn_gen, parrots, conn_spec="all_to_all")
    nest.Connect(parrots, post, conn_spec="all_to_all",
    syn_spec={
        **inh_syn_params,
       # "receptor_type": 2 # Inhibitory receptor
    })





Jan 26 17:10:21 ConnectionManager [Warning]: 
    New connections created, connection descriptors previously obtained using 
    'GetConnections' are now invalid.


In [ ]:
#==================== Simulate and Record Data ====================

multimeter = nest.Create("multimeter", params={
    "record_from": ["V_m", "g_ex", "g_in"],
    "interval": 0.1  # Resolución temporal alta para ver las corrientes
})
nest.Connect(multimeter, post)

nest.Simulate(50000.0) 
events = nest.GetStatus(multimeter, "events")[0] #Obtencion de datos del multimeter

t = events["times"]
V = events["V_m"]
gE = events["g_ex"]
gI = events["g_in"]

E_ex = neuron_params["E_ex"]
E_in = neuron_params["E_in"]

I_E = gE * (-E_ex + V)
I_I = gI * (- E_in + V)
I_net = I_E + I_I


In [ ]:
#  Definimos los 3 intervalos de tiempo de observacion
intervalos = [
    (0, 1000),      # Intervalo 1
    (29000, 30000), # Intervalo 2 
    (49000, 50000)  # Intervalo 3
]
fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharey=True)

for ax, (inicio, fin) in zip(axs, intervalos):
    ax.axhline(0, color='black', linestyle='-', linewidth=2, alpha=0.7)
    ax.plot(t, I_E/1000, color="black", label="Excitatory")
    ax.plot(t, I_I/1000, color="gray", label="Inhibitory")
    ax.plot(t, I_net/1000, color="green", label="Net")
    
    ax.set_xlim(inicio, fin)
    ax.set_ylim(-10,10)
    ax.set_ylabel("Synaptic current (nA)")
    ax.legend(frameon=False, loc="upper right")
    
 
    #ax.set_title(f"Intervalo: {inicio} - {fin} ms")

axs[-1].set_xlabel("Time (ms)")

plt.tight_layout()
plt.show()


# Fig 1D usando iaf_psc_alpha

In [ ]:
#  Definimos los 3 intervalos de tiempo de o0bservacion
intervalos = [
    (0, 500),      # Intervalo 1
    (500, 1500), # Intervalo 2 
    (29000, 30000)  # Intervalo 3
]
fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharey=True)

for ax, (inicio, fin) in zip(axs, intervalos):
    
    ax.axhline(0, color='black', linestyle='--', linewidth=2, alpha=0.7)
    ax.plot(t, -I_E/1000, color="black", label="Excitatory")
    ax.plot(t, -I_I/1000, color="gray", label="Inhibitory")
    ax.plot(t, -I_net/1000, color="green", label="Net")
    
    ax.set_xlim(inicio, fin)
    ax.set_ylabel("Synaptic current (nA)")
    ax.legend(frameon=False, loc="upper right")
    
 
    #ax.set_title(f"Intervalo: {inicio} - {fin} ms")

axs[-1].set_xlabel("Time (ms)")

plt.tight_layout()
plt.show()
